In [1]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

True

In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


In [6]:
df = pd.read_csv('/content/drive/MyDrive/Womens Clothing E-Commerce Reviews.csv')

In [ ]:
df.columns

Index(['Unnamed: 0', 'Clothing ID', 'Age', 'Title', 'Review Text', 'Rating',
       'Recommended IND', 'Positive Feedback Count', 'Division Name',
       'Department Name', 'Class Name'],
      dtype='object')

In [7]:
df = df[['Review Text', 'Rating']]

In [8]:
df['sentiment'] = df['Rating'].apply(lambda x: 1 if x > 3 else 0)

In [9]:
df = df[['Review Text', 'sentiment']]

In [10]:
df = df.dropna()

In [11]:
X = df['Review Text'].values
y = df['sentiment'].values

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X)
X_tokenized = tokenizer.texts_to_sequences(X)

maxlen = 100
X_padded = pad_sequences(X_tokenized, padding='post', maxlen=maxlen)

X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D


vocab_size = 5000
embedding_dim = 100
maxlen = 100

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=maxlen))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(units=100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [13]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), verbose=1)


Epoch 1/10
283/283 ━━━━━━━━━━━━━━━━━━━━ 68s 230ms/step - accuracy: 0.7662 - loss: 0.5596 - val_accuracy: 0.7693 - val_loss: 0.5395
Epoch 2/10
283/283 ━━━━━━━━━━━━━━━━━━━━ 62s 218ms/step - accuracy: 0.7658 - loss: 0.5386 - val_accuracy: 0.7693 - val_loss: 0.5302
Epoch 3/10
283/283 ━━━━━━━━━━━━━━━━━━━━ 83s 223ms/step - accuracy: 0.7700 - loss: 0.5258 - val_accuracy: 0.7693 - val_loss: 0.5381
Epoch 4/10
283/283 ━━━━━━━━━━━━━━━━━━━━ 83s 227ms/step - accuracy: 0.7761 - loss: 0.5298 - val_accuracy: 0.7693 - val_loss: 0.5058
Epoch 5/10
283/283 ━━━━━━━━━━━━━━━━━━━━ 88s 250ms/step - accuracy: 0.7856 - loss: 0.4936 - val_accuracy: 0.8594 - val_loss: 0.3400
Epoch 6/10
283/283 ━━━━━━━━━━━━━━━━━━━━ 76s 231ms/step - accuracy: 0.8622 - loss: 0.3183 - val_accuracy: 0.8892 - val_loss: 0.2689
Epoch 7/10
283/283 ━━━━━━━━━━━━━━━━━━━━ 82s 232ms/step - accuracy: 0.9084 - loss: 0.2300 - val_accuracy: 0.8918 - val_loss: 0.2780
Epoch 8/10
283/283 ━━━━━━━━━━━━━━━━━━━━ 82s 232ms/step - accuracy: 0.9177 - loss: 0

In [14]:
# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Accuracy: {accuracy:.4f}')


142/142 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.8775 - loss: 0.3256
Test Accuracy: 0.8841


In [ ]:
# 142/142 ━━━━━━━━━━━━━━━━━━━━ 7s 47ms/step - accuracy: 0.8453 - loss: 0.3677
# Test Accuracy: 0.8552 5epochs
# 142/142 ━━━━━━━━━━━━━━━━━━━━ 5s 35ms/step - accuracy: 0.7677 - loss: 0.5430
# Test Accuracy: 0.7693

In [ ]:
import pickle

# Save the tokenizer
with open('/content/drive/MyDrive/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save the model
model.save('/content/drive/MyDrive/sentiment_analysis_model.h5')


In [ ]:
# prompt: how to load the above save model and do prediction

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

# Load the tokenizer
with open('/content/drive/MyDrive/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Load the saved model
model = tf.keras.models.load_model('/content/drive/MyDrive/sentiment_analysis_model.h5')

# Example new text for prediction
new_text = ["This product is amazing!", "I'm so disappointed with this."]

# Tokenize and pad the new text
new_text_tokenized = tokenizer.texts_to_sequences(new_text)
new_text_padded = pad_sequences(new_text_tokenized, padding='post', maxlen=100)

# Make predictions
predictions = model.predict(new_text_padded)

# Print the predictions
for i, text in enumerate(new_text):
    print(f"Text: {text}")
    if predictions[i] > 0.5:
        print("Sentiment: Positive")
    else:
        print("Sentiment: Negative")


In [ ]:
# Example of making predictions
new_reviews = ['This is the best product ever!', 'I will never buy this again.']
new_reviews_tokenized = tokenizer.texts_to_sequences(new_reviews)
new_reviews_padded = pad_sequences(new_reviews_tokenized, padding='post', maxlen=maxlen)

# Predict
predictions = model.predict(new_reviews_padded)
predictions = ['positive' if p > 0.5 else 'negative' for p in predictions]

print(predictions)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step
['negative', 'positive']
